# Training a sequence tagger for Slot Filling

<table>
  <td>
    <a target="_blank" href="https://www.recogn.ai/biome-text/documentation/tutorials/Training_a_sequence_tagger_for_Slot_Filling.html"><img src="https://www.recogn.ai/biome-text/assets/img/biome-isotype.svg" width=32 />View on recogn.ai</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/Training_a_sequence_tagger_for_Slot_Filling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/Training_a_sequence_tagger_for_Slot_Filling.ipynb"><img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=32 />View source on GitHub</a>
  </td>
</table>

In this tutorial we will train a sequence tagger for filling slots in spoken requests.
The goal is to look for specific pieces of information in the request and tag the corresponding tokens accordingly. 
The requests will include several intents, from getting weather information to adding a song to a playlist, each requiring its own set of slots.
Therefore, slot filling often goes hand in hand with intent classification.
In this tutorial, however, we will only focus on the slot filling task.

Slot filling is closely related to [Named-entity recognition (NER)](https://en.wikipedia.org/wiki/Named-entity_recognition) and the model of this tutorial can also be used to train a NER system.

In this tutorial we will use the [SNIPS data set](https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines) adapted by [Su Zhu](https://github.com/sz128/slot_filling_and_intent_detection_of_SLU/tree/master/data/snips) and our simple [data preparation notebook](https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/data_prep.ipynb).

When running this tutorial in Google Colab, make sure to install *biome.text* first:

In [1]:
!pip install -U git+https://github.com/recognai/biome-text.git

Ignore warnings and don't forget to restart your runtime afterwards (*Runtime -> Restart runtime*).

## Explore the data

Let's take a look at the data before starting with the configuration of our pipeline.
For this we create a `DataSource` instance providing a path to our data.

In [2]:
from biome.text.data import DataSource

In [3]:
train_ds = DataSource(source="https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/train.json")
train_ds.head()

,text,labels,intent,path
0,"[Find, the, schedule, for, Across, the, Line, ...","[O, O, B-object_type, O, B-movie_name, I-movie...",SearchScreeningEvent,https://biome-tutorials-data.s3-eu-west-1.amaz...
1,"[play, Party, Ben, on, Slacker]","[O, B-artist, I-artist, O, B-service]",PlayMusic,https://biome-tutorials-data.s3-eu-west-1.amaz...
2,"[play, a, 1988, soundtrack]","[O, O, B-year, B-music_item]",PlayMusic,https://biome-tutorials-data.s3-eu-west-1.amaz...
3,"[Can, you, play, The, Change, Is, Made, on, Ne...","[O, O, O, B-track, I-track, I-track, I-track, ...",PlayMusic,https://biome-tutorials-data.s3-eu-west-1.amaz...
4,"[what, is, the, forecast, for, colder, in, Ans...","[O, O, O, O, O, B-condition_temperature, O, B-...",GetWeather,https://biome-tutorials-data.s3-eu-west-1.amaz...
5,"[What's, the, weather, in, Totowa, WY, one, mi...","[O, O, O, O, B-city, B-state, B-timeRange, I-t...",GetWeather,https://biome-tutorials-data.s3-eu-west-1.amaz...
6,"[Play, a, tune, from, Space, Mandino, .]","[O, O, B-music_item, O, B-artist, I-artist, O]",PlayMusic,https://biome-tutorials-data.s3-eu-west-1.amaz...
7,"[give, five, out, of, 6, stars, to, current, e...","[O, B-rating_value, O, O, B-best_rating, B-rat...",RateBook,https://biome-tutorials-data.s3-eu-west-1.amaz...
8,"[Play, some, chanson, style, music.]","[O, O, B-genre, O, O]",PlayMusic,https://biome-tutorials-data.s3-eu-west-1.amaz...
9,"[I, would, give, French, Poets, and, Novelists...","[O, O, O, B-object_name, I-object_name, I-obje...",RateBook,https://biome-tutorials-data.s3-eu-west-1.amaz...


As we can see we have two relevant columns for our task: *text* and *labels*. 
The *intent* column will be ignored in this tutorial. 
The *path* column is added automatically by the [DataSource](../../api/biome/text/data/datasource.html#datasource) class to keep track of the source file.

The input already comes pre-tokenized and each token in the *text* column has a label/tag in the *labels* column, this means that both list always have the same length.
The labels are given in the [BIO tagging scheme](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), which is widely used in Slot Filling/NER systems.

When specifying the [TokenClassification](../../api/biome/text/modules/heads/token_classification.html#tokenclassification) head (see [below](#Configure-your-biome.text-Pipeline)), the tokenization step in the pipeline is automatically disabled and the input is expected to be a list of tokens.

The [DataSource](../../api/biome/text/data/datasource.html#datasource) class stores the data in an underlying [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) that you can easily access.
For example, let's check the size of our training data:

In [4]:
len(train_ds.to_dataframe())

13084

Or let's check how many different labels/tags we have:

In [5]:
df = train_ds.to_dataframe().compute()
labels_total = df.labels.sum()
len(set(labels_total))

72

and how they are distributed:

In [6]:
import pandas as pd
pd.Series(labels_total).value_counts()

O                               59610
I-object_name                    7400
I-playlist                       3230
B-object_type                    3023
B-object_name                    2778
                                ...  
I-cuisine                          28
I-facility                         14
I-object_select                     3
I-object_part_of_series_type        3
I-playlist_owner                    1
Length: 72, dtype: int64

::: tip

The [TaskHead](../../api/biome/text/modules/heads/task_head.html#taskhead) of our model (the [TokenClassification](../../api/biome/text/modules/heads/token_classification.html#tokenclassification)) will expect a *text* and a *labels* column to be present in the dataframe. 
Since they are already present, there is no need for a `mapping` in the [DataSource](../../api/biome/text/data/datasource.html#datasource).

:::

## Configure your *biome.text* Pipeline

A typical [Pipeline](../../api/biome/text/pipeline.html#pipeline) consists of tokenizing the input, extracting features, applying a language encoding (optionally) and executing a task-specific head in the end.

After training a pipeline, you can use it to make predictions or explore the underlying model via the [explore UI](../../documentation/user-guides/02.explore.html).

As a first step we must define a configuration for our pipeline. 
In this tutorial we will create a configuration dictionary and use the `Pipeline.from_config()` method to create our pipeline, but there are [other ways](../../api/biome/text/pipeline.html#pipeline).

A *biome.text* pipeline has the following main components:

```yaml
name: # a descriptive name of your pipeline

tokenizer: # how to tokenize the input

features: # input features of the model

encoder: # the language encoder

head: # your task configuration

```

See the [Configuration section](../../documentation/user-guides/05.configuration.html) for a detailed description of how these main components can be configured.

Our complete configuration for this tutorial will be following:

In [7]:
pipeline_dict = {
    "name": "slot_filling",
    "features": {
        "word": {
            "embedding_dim": 300,
            "weights_file": "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip",
        },
        "char": {
            "embedding_dim": 32,
            "encoder": {
                "type": "gru",
                "bidirectional": True,
                "num_layers": 1,
                "hidden_size": 32,
            },
            "dropout": 0.1,
        },
    },
    "encoder": {
        "type": "gru",
        "hidden_size": 128,
        "num_layers": 1,
        "bidirectional": True,
    },
    "head": {
        "type": "TokenClassification",
        "labels": list(set(labels_total)),
        "label_encoding": "BIO",
        "feedforward": {
            "num_layers": 1,
            "hidden_dims": [128],
            "activations": ["relu"],
            "dropout": [0.1],
        },
        "dropout": 0.1,
    },
}

As you can see in the `features` section, we will use pretrained word embeddings from [fasttext](https://fasttext.cc/docs/en/english-vectors.html) for our `word` features.
Keep in mind that your `embedding_dim` parameter must be equal to the dimensions of those pretrained embeddings.

With the dictionary above we can now create a `Pipeline`.

In [8]:
from biome.text import Pipeline

In [9]:
pl = Pipeline.from_config(pipeline_dict)

## Create a vocabulary

Before we can start the training we need to create the vocabulary for our model.
For this we define a `VocabularyConfiguration`.

In [10]:
from biome.text import VocabularyConfiguration

Since we use pretrained word embeddings we will also consider the validation data when creating the vocabulary. 

In [11]:
valid_ds = DataSource(source="https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/valid.json")

We also get rid of the rarest words by adding the `min_count` argument and set it to 2 for the word feature vocabulary.
For a complete list of available arguments see the [VocabularyConfiguration API](../../api/biome/text/configuration.html#vocabularyconfiguration).

In [12]:
vocab_config = VocabularyConfiguration(
    sources=[train_ds, valid_ds],
    min_count={"word": 2},
)

We then pass this configuration to our `Pipeline` to create the vocabulary.
Apart from the loading bar of building the vocabulary, there will be two more loading bars corresponding to the `weights_file` provided in the word feature:
 - the progress of downloading the file (this file will be cached)
 - the progress loading the weights from the file 

In [13]:
pl.create_vocabulary(vocab_config)

After creating the vovocab_configbulary we can check the size of our entire model in terms of trainable parameters:

In [14]:
pl.trainable_parameters

1989112

## Configure the trainer

As a next step we have to configure the *trainer*.

For this tutorial we will use the default configuration that has sensible values and works alright for our experiment.
You can have a look at the [TrainerConfiguration API](../../api/biome/text/configuration.html#trainerconfiguration) for a complete list of available configurations.

::: tip

In case you have a cuda device available, you also specify it here.

:::

In [15]:
from biome.text import TrainerConfiguration

In [18]:
trainer_config = TrainerConfiguration()

## Train your model

Now we have everything ready to start the training of our model:
- training data set
- vocabulary
- trainer

Apart from the validation data source to estimate the generalization error, we will also pass in a test data set in case we want to do some Hyperparameter optimization and compare different encoder architectures in the end. 
For this we will create another `DataSource` pointing to our test data.

In [19]:
test_ds = DataSource(source="https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/test.json")

The training output will be saved in a folder specified by the `output` argument of the `train` method. 
It will contain the trained model weights and the metrics, as well as the vocabulary and a *log* folder for visualizing the training process with [tensorboard](https://www.tensorflow.org/tensorboard/).

When the training has finished it will automatically make a pass over the test data with the best weights to gather the test metrics.

In [ ]:
pl.train(
    output="output",
    training=train_ds,
    validation=valid_ds,
    test=test_ds,
    trainer=trainer_config,
)

The model above achieves an overall F1 score of around **0.95**, which is not bad when compared to [published results](https://nlpprogress.com/english/intent_detection_slot_filling.html) of the same data set.
You could continue the experiment changing the encoder to an LSTM network, try out a transformer architecture or fine tune the trainer.
But for now we will go on and make our first predictions with this trained model.

## Make your first predictions

Now that we trained our model we can go on to make our first predictions.
First we must load our trained model into a new `Pipeline`:

In [22]:
pl_trained = Pipeline.from_pretrained("output/model.tar.gz")

We then provide the input expected by our `TaskHead` of the model to the `Pipeline.predict()` method.
In our case it is a `TokenClassification` head that classifies a `text` input. **Remember that the input has to be pre-tokenized!**

In [89]:
text = "can you play biome text by backstreet recognais on Spotify".split()
prediction = pl_trained.predict(text)
list(zip(text, prediction["tags"]))

[('can', 'O'),
 ('you', 'O'),
 ('play', 'O'),
 ('biome', 'B-track'),
 ('text', 'I-track'),
 ('by', 'O'),
 ('backstreet', 'B-artist'),
 ('recognais', 'I-artist'),
 ('on', 'O'),
 ('Spotify', 'B-service')]

Apart from the most likely *tags*, the `prediction` dictionary contains the *logits* and *probs* of each of the label for each of the input token.